In [169]:
# STD
import codecs
from functools import reduce
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import time
import math
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import subprocess

In [177]:
batch_size = 64
embedding_dims = 100
hidden_dims = embedding_dims*2
en_file = "data/train/train_bpe.en"
fr_file = "data/train/train_bpe.fr"
num_epochs = 10
learning_rate = 0.01
max_sentence_len = 50
cuda = False


# To Do

    -Currently has 17k vocab size for french and 15k for english, smaller? seems fast enough... not neccesairy?
    
    
# usefulll for later

    -That is correct. Simply set ignore_index to your padding index. Also set size_average to false and divide the loss by the number of non padding tokens. <<--- someone said this, but cant you just let loss itself avarage it out by not setting it to false? thats what i did now, might try some runs with this method when we have an evaluation method to check for difference.
    
# questions


In [171]:
"""
Module defining class and function relevant for data I/O and preprocessing.
"""

# STD
import codecs
from functools import reduce
from collections import defaultdict

# EXT
import numpy as np
import torch
from torch.utils.data import Dataset


class ParallelCorpus(Dataset):
    """
    Class that contains a parallel corpus used for training IBM Model 1 and 2.
    """
    def __init__(self, source_path, target_path, max_sentence_length=50, max_source_vocab_size=np.inf,
                 max_target_vocab_size=np.inf, use_indices_from=None):
        if use_indices_from:
            assert type(use_indices_from) == type(self), "You can only use indices from another ParallelCorpus class"

        # Enable a way to read multiple paths into one corpus if wanted
        source_paths = source_path if self.is_listlike(source_path) else (source_path, )
        target_paths = target_path if self.is_listlike(target_path) else (target_path, )

        # Read in all the data
        self.max_sentence_length = max_sentence_length
        self.source_sentences = self.read_all(source_paths)
        self.target_sentences = self.read_all(target_paths)
        self.size = len(self.source_sentences)

        # Index corpus and create vocabulary
        indexed_source = self.index_corpus(
            self.source_sentences, max_vocab_size=max_source_vocab_size,
            given_word2idx=None if not use_indices_from else use_indices_from.source_w2i
        )
        self.source_idx, self.source_vocab, self.source_w2i, self.source_i2w, self.source_vocab_size, self.source_lengths = indexed_source
        indexed_target = self.index_corpus(
            self.target_sentences, add_sentence_delimiters=True, max_vocab_size=max_target_vocab_size,
            given_word2idx=None if not use_indices_from else use_indices_from.target_w2i
        )
        self.target_idx, self.target_vocab, self.target_w2i, self.target_i2w, self.target_vocab_size, self.target_lengths = indexed_target

        # Create position vectors
        self.pos_base = [i for i in range(1, max_sentence_length + 1)]
        self.positions = [self.pos_base[:i] for i in self.source_lengths]
        self.source_pad = self.source_w2i['<pad>']
        self.target_pad = self.target_w2i['<pad>']

        # Pad the sentences and positions - useful in order to convert everything to tensors and train batch
        self.source_padded = self.pad_sequences(self.source_idx, self.source_lengths, self.source_w2i['<pad>'])
        self.target_padded = self.pad_sequences(self.target_idx, self.target_lengths, self.target_w2i['<pad>'])
        self.positions = self.pad_sequences(self.positions, self.source_lengths, 0)
        self.source_sentence_ids = np.array(range(len(self.source_sentences)))
        self.target_sentence_ids = np.array(range(len(self.target_sentences)))

        # Convert everything into tensors for pytorch and sort by sentence size, descending
        data = [
            self.source_padded, self.target_padded, self.source_lengths, self.target_lengths, self.positions,
            self.source_sentence_ids, self.target_sentence_ids
        ]
        tensors = list(map(self.convert_to_tensor, data))
        sorted_tensors = self.sort_tensors(*tensors)
        self.source_tensor, self.target_tensor, self.source_lengths, self.target_lengths, self.positions, \
            self.source_sentence_ids, self.target_sentence_ids = sorted_tensors

    def read_all(self, paths):
        """
        Read all corpus file at given paths and merge results.
        """
        def _combine_lists(a, b):
            a.extend(b)
            return a

        sentences_per_path = [self.read_corpus_file(path, self.max_sentence_length) for path in paths]
        return reduce(_combine_lists, sentences_per_path)

    @staticmethod
    def read_corpus_file(path, max_sentence_length, filter_characters=[]):
        sentences = []

        with codecs.open(path, "rb", "utf-8") as corpus:
            for line in corpus.readlines():
                tokens = [token for token in line.strip().split() if token not in filter_characters]

                # Filter out sentences that are too long
                if len(tokens) <= max_sentence_length:
                    sentences.append(tokens)

        return sentences

    def index_corpus(self, sentences, max_vocab_size=np.inf, add_sentence_delimiters=False, given_word2idx=None):
        # Use indices create on other set if given
        word2idx = self.init_word2idx() if not given_word2idx else given_word2idx
        indexed_sentences = []
        sentence_lengths = []
        token_freqs = defaultdict(int)

        for line in sentences:
            current_indexed_sentence = [] if not add_sentence_delimiters else [word2idx["<bos>"]]

            for word in line:
                token_freqs[word] += 1
                current_indexed_sentence.append(word2idx[word])

            if add_sentence_delimiters:
                current_indexed_sentence.append(word2idx["<eos>"])

            indexed_sentences.append(current_indexed_sentence)

            sentence_length = len(line) if not add_sentence_delimiters else len(line) + 2
            sentence_lengths.append(sentence_length)

        # Remove infrequent words (they will be mapped to <unk> later) until max_vocab size is reached
        vocab_size = len(word2idx)
        sorted_token_freqs = sorted(list(token_freqs.items()), key=lambda x: x[1])

        while len(sorted_token_freqs) > max_vocab_size:
            infrequent_word, _ = sorted_token_freqs.pop(0)
            del word2idx[infrequent_word]

        vocab = set(word2idx.keys())

        idx2word = defaultdict(lambda: "<unk>", {idx: w for (w, idx) in word2idx.items()})

        # After reading the data, unknown word just return the index of the <unk> token (don't generate new indices)
        word2idx = defaultdict(lambda: word2idx["<unk>"], word2idx)

        return indexed_sentences, vocab, word2idx, idx2word, vocab_size, sentence_lengths

    @staticmethod
    def init_word2idx():
        word2idx = defaultdict(lambda: len(word2idx))
        _ = word2idx["<pad>"], word2idx["<unk>"], word2idx["<bos>"], word2idx["<eos>"]
        return word2idx

    @staticmethod
    def pad_sequences(sentence_idx, seq_lengths, pad):
        # Fill everything with padding first
        padding = np.full((len(sentence_idx), max(seq_lengths)), pad)

        # Replace with actual token ids wherever possible
        for idx, (seq, seqlen) in enumerate(zip(sentence_idx, seq_lengths)):
            padding[idx, :seqlen] = seq

        return padding

    @staticmethod
    def convert_to_tensor(data):
        if torch.cuda.is_available():
            return torch.LongTensor(data).cuda()
        return torch.LongTensor(data)

    @staticmethod
    def sort_tensors(source, target, source_lengths, target_lengths, pos, source_ids, target_ids):
        target_lengths, perm_idx = target_lengths.sort(0, descending=True)
        source_tensor = source[perm_idx]
        target_tensor = target[perm_idx]
        source_lengths = source_lengths[perm_idx]
        positions = pos[perm_idx]
        source_ids = source_ids[perm_idx]
        target_ids = source_ids[perm_idx]
        return source_tensor, target_tensor, source_lengths, target_lengths, positions, source_ids, target_ids

    def __len__(self):
        return self.source_tensor.size(0)

    def __getitem__(self, index):
        return self.source_tensor[index], self.target_tensor[index], self.source_lengths[index], \
               self.target_lengths[index], self.positions[index]

    @staticmethod
    def is_listlike(obj):
        return type(obj) in (tuple, list, set)


In [172]:
dataset = ParallelCorpus(fr_file, en_file, max_sentence_len, cuda)

In [173]:
data_loader = DataLoader(dataset, batch_size=batch_size)

# run from here

In [174]:
class Decoder(nn.Module):
    def __init__(self, source_vocab_size, max_sentence_len, target_vocab_size, embedding_dims):
        super(Decoder, self).__init__()
        
        # Encoder stuff
        self.encoder_embeddings = nn.Embedding(source_vocab_size, embedding_dims) #embed words
        self.pos_embeddings = nn.Embedding(max_sentence_len+1, embedding_dims) #embed positions
        
        # Decoder stuff
        self.decoder_embeddings = nn.Embedding(target_vocab_size, embedding_dims)
        self.lstm = nn.LSTM(embedding_dims, hidden_dims, batch_first=True)
        self.scale_h0 = nn.Linear(embedding_dims*2, hidden_dims)
        self.out = nn.Linear(embedding_dims*2 + hidden_dims, target_vocab_size)
        
        # Attention stuff
        
        # used only in the concat version
        self.attn_layer = nn.Linear(embedding_dims*2 + hidden_dims, 1)
        
        self.attn_soft = nn.Softmax(dim=2)
        
    def forward(self, target_sentences, target_lengths, source_lengths, source_sentences, positions, max_len):
        words = self.decoder_embeddings(target_sentences)
        
        
        encoder_outputs = encoder(source_sentences, positions)
        
        # avg out encoder data to use as hidden state
        avg = torch.mean(encoder_outputs,1)
        hidden = self.scale_h0(avg)
        
        # get hidden state from encoder RNN
        packed_input = pack_padded_sequence(words, target_lengths, batch_first=True)
        packed_output, (ht, ct) = self.lstm(packed_input, (torch.unsqueeze(hidden,0),torch.unsqueeze(hidden,0)))
        lstm_out, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        # prepare lstm Hidden layers for input into attention,
        # we add the hidden layer as zeroth lstm_out and remove the last one
        # this is because we need h_i-1, not h_i
        lstm_to_attn = torch.cat((torch.unsqueeze(hidden,1),lstm_out),1)
        lstm_to_attn = lstm_to_attn = lstm_to_attn[:,:-1,:]
        
        context = self.attention(lstm_to_attn, encoder_outputs, source_lengths, max_len)
        
        # combine with non existing context vectors
        combined = torch.cat((lstm_out,context),2)
        out = self.out(combined)
        return out
    
    def attention(self, lstm_to_attn, encoder_outputs, source_lengths, max_len):
                
        # repeat the lstm out in third dimension and the encoder outputs in second dimension so we can make a meshgrid
        # so we can do elementwise mul for all possible combinations of h_j and s_i
        h_j = encoder_outputs.unsqueeze(1).repeat(1,lstm_to_attn.size(1),1,1)
        s_i = lstm_to_attn.unsqueeze(2).repeat(1,1,encoder_outputs.size(1),1)
        
        # get the dot product between the two to get the energy
        # the unsqueezes are there to emulate transposing. so we can use matmul as torch.dot doesnt accept matrices
        energy = s_i.unsqueeze(3).matmul(h_j.unsqueeze(4)).squeeze(4)
        
#         # this is concat attention, its a different form then the ones we need
#         cat = torch.cat((s_i,h_j),3)
        
#         energy = self.attn_layer(cat)

        # reshaping the encoder outputs for later
        encoder_outputs = encoder_outputs.unsqueeze(1)
        encoder_outputs = encoder_outputs.repeat(1,energy.size(1),1,1)
    
        # apply softmax to the energys 
        allignment = self.attn_soft(energy)
        
        # create a mask like : [1,1,1,0,0,0] whos goal is to multiply the attentions of the pads with 0, rest with 1
        idxes = torch.arange(0,max_len,out=max_len).unsqueeze(0)
        mask = Variable((idxes<source_lengths.unsqueeze(1)).float())
        
        # format the mask to be same size() as the attentions
        mask = mask.unsqueeze(1).unsqueeze(3).repeat(1,allignment.size(1),1,1)
        
        # apply mask
        masked = allignment * mask
        
        # now we have to rebalance the other values so they sum to 1 again
        # this is done by dividing each value by the sum of the sequence
        # calculate sums
        msum = masked.sum(-2).repeat(1,1,masked.size(2)).unsqueeze(3)
        
        # rebalance
        attentions = masked.div(msum)
        
        # now we shape the attentions to be similar to context in size
        allignment = allignment.repeat(1,1,1,encoder_outputs.size(3))

        # make context vector by element wise mul
        context = attentions * encoder_outputs
        
        context2 = torch.sum(context,2)
        
        return context2
    
    def encoder(self, source_sentences, positions):
        words = self.word_embeddings(source_sentences)
        pos = self.pos_embeddings(positions)
        return torch.cat((words,pos),2)
        
        

In [175]:
# initialiase models, loss and optimizer
if (cuda):
    decoder = Decoder(dataset.source_vocab_size, max_sentence_len, dataset.target_vocab_size, embedding_dims).cuda()
    loss_function = nn.CrossEntropyLoss(ignore_index = dataset.target_pad).cuda()
else:
    decoder = Decoder(dataset.source_vocab_size, max_sentence_len, dataset.target_vocab_size, embedding_dims).cpu()
    loss_function = nn.CrossEntropyLoss(ignore_index = dataset.target_pad).cpu()
    
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), learning_rate)

In [176]:
epoch_losses = []
iterations = len(data_loader)

for epoch in range(0, num_epochs):
    start = time.time()
    batch_losses = []
    batch = 0
    for source_batch, target_batch, source_lengths, target_lengths, batch_positions in data_loader:
        batch_start = time.time()
        max_len = source_lengths.max()
        source_batch = source_batch[:,:max_len]
        batch_positions = batch_positions[:,:max_len]
        
        source_batch = torch.autograd.Variable(source_batch)
        target_batch = torch.autograd.Variable(target_batch)
        source_lengths = torch.autograd.Variable(source_lengths)
        target_lengths = torch.autograd.Variable(target_lengths)
        batch_positions = torch.autograd.Variable(batch_positions)
        
        if (cuda):
            source_batch = source_batch.cuda()
            target_batch = target_batch.cuda()
            source_lengths = source_lengths.cuda()
            target_lengths = target_lengths.cuda()
            batch_positions = batch_positions.cuda()
            
            
        # get encoder and decoder outputs
        # encoder_out = encoder(source_batch, batch_positions)
        decoder_out = decoder(target_batch, target_lengths, source_lengths, source_batch, batch_positions, max_len)
        
        # remove the start token from the targets and the end token from the decoder
        decoder_out2 = decoder_out[:,:-1,:]
        target_batch2 = target_batch[:,1:decoder_out.size(1)]
        
        # pytorch expects the inputs for the loss function to be: (batch x classes)
        # cant handle sentences so we just transform our data to treath each individual word as a batch:
        # so new batch size = old batch * padded sentence length
        # should not matter as it all gets averaged out anyway
        decoder_out3 = decoder_out2.contiguous().view(decoder_out2.size(0)*decoder_out2.size(1),decoder_out2.size(2))
        target_batch3 = target_batch2.contiguous().view(target_batch2.size(0)*target_batch2.size(1))
        
        # calculate loss
        #print(decoder_out2.size())
        #print(target_batch[0])
#         _, sentence = torch.max(decoder_out2[0],1)
#         test_pred = [dataset.target_i2w[word] for word in sentence.cpu().numpy()]
#         print(test_pred)
#         test_real = [dataset.target_i2w[word] for word in target_batch2[0].cpu().numpy()]
#         print(test_real)
        
        loss = loss_function(decoder_out3,target_batch3)
        batch_losses.append(loss)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_time = time.time() - batch_start 
        print('\r[Epoch {:03d}/{:03d}] Batch {:06d}/{:06d} [{:.1f}/s] '.format(epoch+1, num_epochs, batch+1, iterations, batch_time), end='')
        batch += 1
        
    avg_loss = sum(batch_losses) / iterations 
    epoch_losses.append(avg_loss)
    print('Time: {:.1f}s Loss: {:.3f} score2?: {:.6f}'.format(time.time() - start, avg_loss, 0))


RuntimeError: Expected object of type torch.LongTensor but found type torch.cuda.LongTensor for argument #3 'index'

In [156]:
torch.save(decoder, "decoder30bpe.model")

C:\Users\Rasyan\Anaconda3\lib\site-packages\torch\serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [166]:
def evaluate(model, eval_set, target_path, reference_file_path):
    data_loader = DataLoader(eval_set, batch_size=5)
    softmax = nn.Softmax(dim=2)
    idx2word = evaluation_set.target_i2w
    sorted_sentence_ids = evaluation_set.target_sentence_ids.cpu().numpy()
    translated_sentences = []

    # Decode
    #for source_batch, target_batch, source_lengths, target_lengths, batch_positions in data_loader:
    for source_batch, target_batch, source_lengths, target_lengths, batch_positions in data_loader:
        # TODO: Don't use target sentences for prediction!
        max_len = source_lengths.max()
        output = model(target_batch, target_lengths, source_lengths, source_batch, batch_positions, max_len)
        normalized_output = softmax.forward(output)
        predictions = normalized_output.max(2)[1].cpu().numpy()  # Only get indices
        _, sentence = torch.max(normalized_output[0],1)
        print([idx2word[word] for word in sentence.cpu().numpy()])
        print([idx2word[word] for word in target_batch[0].cpu().numpy()])
        print(" ")
        
        for sentence_index in range(predictions.shape[0]):
            token_indices = predictions[sentence_index]
            tokens = list(map(lambda idx: idx2word[idx], token_indices))

            eos_index = len(tokens)
            if "<eos>" in tokens:
                eos_index = tokens.index("<eos>")

            tokens = tokens[:eos_index]  # Cut off after first end of sentence token
            translated_sentence = " ".join(tokens).replace("@@ ", "")
            translated_sentences.append(translated_sentence)
            #print(translated_sentence)
            #print([idx2word[word] for word in target_batch[sentence_index].cpu().numpy()])
            #print(" ")
            
            

    # Bring sentence back into the order they were in the test set
    print(translated_sentences[0])
    translated_sentences = np.array(translated_sentences)[sorted_sentence_ids]
    print(translated_sentences[0])
    

    # Write to file
    with codecs.open(target_path, "wb", "utf-8") as target_file:
        for sentence in translated_sentences:
            target_file.write("{}\n".format(sentence))

    out = subprocess.getoutput(
        "perl ./multi-bleu.perl {} < {}".format(reference_file_path, target_path)
    )
    print(out[out.index("BLEU"):])

In [168]:
model = torch.load("./decoder30bpe.model")
max_allowed_sentence_len = 50
training_set = ParallelCorpus(
    source_path="./data/train/train_bpe.fr", target_path="./data/train/train_bpe.en",
    max_sentence_length=max_allowed_sentence_len
)
evaluation_set = ParallelCorpus(
    source_path="./data/test/test_2017_flickr_bpe.fr", target_path="./data/test/test_2017_flickr_bpe.en",
    max_sentence_length=max_allowed_sentence_len, use_indices_from=training_set
)
evaluate(
    model, evaluation_set, target_path="./eval_out.txt",
    reference_file_path="./data/test/test_2017_flickr_truecased.en"
)


C:\Users\Rasyan\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


['behind', 'a', 'white', 'festival', 'dark', 'dogs', 'with', 'at', 'a', '.', 'a', 'row', 'amount', '.', 'a', 'white', 'of', 'a', 'row', '.', 'a', 'row', '.', 'a', '.', 'one', 'a', 'row', 'sign', 'above', 'by', 'a', 'other', 'other', 'dog', 'yellow', 'gate', 'point', 'a', 'woman', 'row', '.', '<eos>', 'white']
['<bos>', 'behind', 'a', 'bar@@', 'b', 'w@@', 'ired', 'fence', ',', 'there', 'is', 'a', 'large', 'ad', 'on', 'the', 'side', 'of', 'the', 'building', 'above', 'a', 'row', 'of', 'cars', ',', 'showing', 'a', 'red', 'fox', 'running', 'with', 'two', 'small', 'red', ',', 'white', ',', 'and', 'dark', 'seum', 'birds', '.', '<eos>']
 
['a', 'cart', 'cart', 'enjoys', 'with', 'on', 'a', 't-shirt', 'of', '.', 'e', '.', 'a', 'skateboarder', '&quot;', '&quot;', '.', '.', 'ra@@', '.', '.', 'S@@', '&quot;', '.', 'Texas', 'message', '.', '&quot;', '.', '.', '&quot;', 'gray@@', 'road', '.', '<eos>', 'gray@@']
['<bos>', 'a', 'white', 'truck', 'transport', 'trailer', 'with', 'one', 'end', 'cr@@', 'um

['girl', 'woman', 'with', 'a', 'stance', 'top', '.', '.', 'rail', '.', '<eos>', 'front', 'of', 'a', 'rail', '<eos>', '<eos>', '<eos>', '<eos>']
['<bos>', 'a', 'woman', 'in', 'a', 'tank', 'top', 'poses', 'with', 'her', 'dog', 'in', 'front', 'of', 'a', 'guardra@@', 'il', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['man', 'man', 'with', 'a', 'surfing', 'a', 'mouth', '.', '.', '.', 'snowy', '.', 'a', 'snowy', '.', '<eos>', '<eos>', '<eos>', '<eos>']
['<bos>', 'a', 'plane', 'with', 'smoke', 'trailing', 'its', 'as@@', 'c@@', 'ent', 'in', 'front', 'of', 'a', 'snow', 'covered', 'mountain', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '

['commuters', 'man', 'and', 'legs', '<eos>', 'the', 'lake', '.', '.', 'going', '.', '.', '.', '.', '<eos>', 'swimming']
['<bos>', 'the', 'feet', 'and', 'legs', 'of', 'a', 'person', 'underwater', 'while', 'swimming', 'through', 'greenish', 'water', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['a', 'group', 'of', 'women', 'jumping', 'over', 'a', 'puddle', '.', '.', '.', '.', 'water', '.', '<eos>', 'bare']
['<bos>', 'a', 'group', 'of', 'women', 'jumping', 'over', 'a', 'hurdle', 'and', 'landing', 'in', 'a', 'puddle', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

['a', 'bicyclist', 'on', 'a', 'mean@@', 'man', '<eos>', 'a', '<eos>', '<eos>', '<eos>', '.', '<eos>', '<eos>']
['<bos>', 'a', 'cyclist', 'on', 'a', 'winding', 'road', 'in', 'a', 'mountain', 'range', 'area', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['a', 'person', 'with', 'a', 'bike', '<eos>', 'a', 'grass', '<eos>', '<eos>', 'green', '.', '<eos>', '<eos>']
['<bos>', 'a', 'person', 'riding', 'a', 'bicycle', 'on', 'the', 'grass', 'wearing', 'a', 'helmet', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['a', 'black

 
['two', 'motorcycles', 'are', 'down', 'a', 'long', '<eos>', '.', 'river', '<eos>', '<eos>', '<eos>']
['<bos>', 'two', 'motorcycles', 'drive', 'on', 'a', 'road', 'along', 'the', 'river', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['an', 'of', 'a', 'on', 'on', 'in', 'a', '.', '.', '.', '<eos>', '<eos>']
['<bos>', 'graffiti', 'of', 'cartoon', 'space', 'scene', 'on', 'bridge', 'seum', 'pass', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['the', 'crowd', 'field', 'is', 'the', 'i

['a', 'climbing', 'on', 'ladder', 'on', 'the', 'roof', '.', '<eos>', '<eos>', '<eos>']
['<bos>', 'man', 'climbs', 'a', 'ladder', 'to', 'the', 'roof', 'top', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['a', 'brightly', 'of', 'with', 'blue', 'chairs', '.', '.', '<eos>', '<eos>', '<eos>']
['<bos>', 'a', 'boat', 'cab', 'with', 'blue', 'chairs', 'before', 'it', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['an', 'white', 'player', 'in', 'the', '.', '<eos>', 'tree

['<bos>', 'group', 'of', 'dancers', 'on', 'a', 'blue', 'stage', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['protesters', 'are', 'on', 'a', 'street', 'walkway', '.', '<eos>', '<eos>']
['<bos>', 'cyclists', 'racing', 'on', 'a', 'paved', 'street', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
 
['protesters', 'are', 'in', 'in', 'a', 'classroom', '.', '<eos>', '<eos>']
['<bos>', 'children', 'play', 'chess', 'in', 'a', 'classroom', '.

BLEU = 0.00, 18.3/1.0/0.0/0.0 (BP=0.749, ratio=0.775, hyp_len=8822, ref_len=11376)
